In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：自定义训练表格回归模型以进行在线预测和可解释性

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_custom_tabular_regression_online_explain.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK来训练和部署一个用于在线预测的自定义表格回归模型，并附带解释。

了解更多关于[Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)和[Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)。

### 目标

在本教程中，您将学习如何使用Vertex AI训练和可解释AI来创建具有解释的自定义表格回归模型。然后，您将学习如何使用Vertex AI预测服务进行带解释的在线预测请求。或者，您可以使用gcloud命令行工具或在云控制台上在线创建自定义模型。

本教程使用以下Vertex AI服务：

- Vertex AI 训练
- Vertex AI 预测
- Vertex 可解释 AI
- Vertex AI 模型资源
- Vertex AI 终端资源

执行的步骤包括：

- 创建一个用于训练TensorFlow模型的Vertex AI自定义作业。
- 查看训练模型的模型评估。
- 设置模型部署时的解释参数。
- 上传训练模型工件和解释作为模型资源。
- 创建一个服务端点资源。
- 将模型资源部署到服务端点资源。
- 进行带解释的预测。
- 取消部署模型资源。

数据集

本教程使用的数据集是【波士顿房价数据集】(https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。您在本教程中使用的数据集版本是来自 TensorFlow SDK 的版本。训练后的模型会预测房屋的中位价格，单位为1K美元。

### 费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用情况生成费用估算。

开始吧

### 为Python安装Vertex AI SDK和其他所需的包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow==2.15.1

重新启动运行时（仅适用于Colab）

为了使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。请等待直到完成后再继续下一步。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 初始化 Vertex AI SDK for Python

要开始使用 Vertex AI，您必须[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

为您的项目和相应的存储桶初始化 Vertex AI SDK for Python。

In [ ]:
import os

import numpy as np
import tensorflow as tf
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持 GPU 的容器映像以及为虚拟机实例分配的 GPU 数量。例如，要在每个虚拟机分配4个 Nvidia Telsa T4 GPU并使用 GPU 容器映像，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器映像。

在您的地区了解更多有关硬件加速器支持的信息请点击 [这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

**注意**：在 GPU 支持之前的 TF 版本（2.3之前）在此教程中加载自定义模型失败。这是一个已知问题，在 TF 2.3 中得到了修复 -- 这是由于在服务函数中生成的静态图操作导致的。如果您在自定义模型中遇到这个问题，请使用带有 GPU 支持的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建的Docker容器镜像。

最新列表请查看[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

最新列表请查看[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-11"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
- `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 内存
- `vCPUs`：数量为\[ 2, 4, 8, 16, 32, 64, 96 \]

**注意**：以下内容不支持用于训练：

- `standard`：2 个 vCPUs
- `highcpu`：2、4 和 8 个 vCPUs

**注意**：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 自定义模型创建和训练

现在你已经准备好开始创建自己的定制模型并对波士顿房价进行预测训练。

###检查培训包

####包布局

在开始培训之前，查看Python包是如何为自定义培训工作而组装的。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行自定义培训工作的Python脚本。*注意*，当在工作人员池规范中引用`trainer/task.py`时，目录斜杠将被替换为点，文件后缀（.py）将被省略（trainer.task）。

####包装配

在下面的单元中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，编写训练脚本*task.py*的内容。

总结一下，该脚本执行以下步骤：

- 从命令行获取保存模型文件的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从TF.Keras内置数据集加载波士顿房价数据集。
- 使用TF.Keras模型API构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 用`args.epochs`指定的epochs训练模型（`fit()`）。
- 将训练后的模型保存到指定的模型目录中（`save(args.model_dir)`）。
- 将每个特征的最大值写入指定的参数文件中`f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_:_+1:])
    x_test[_], _ = scale(x_test[_:_+1:])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶上。

接下来，将培训文件夹打包成压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

### 创建并运行自定义训练任务

为了训练一个自定义模型，您需要执行两个步骤：
1) 创建一个自定义训练任务。
2) 指定您的训练参数并运行任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，包括以下参数：

- `display_name`: 自定义训练任务的可读名称。
- `container_uri`: 训练容器镜像。
- `requirements`: 训练容器镜像的包要求（例如，pandas）。
- `script_path`: 训练脚本的相对路径。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="boston",
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
    location=LOCATION,
)

print(job)

####准备您的训练参数

现在为您的自定义训练容器定义命令行参数：

- `args`：要传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：指定存储模型工件的命令行参数。您可以使用以下任一方法指定工件的存储位置。
      - **方法1**（将`DIRECT`设置为`True`）：将Cloud Storage位置作为命令行参数传递给您的训练脚本。
      - **方法2**（将`DIRECT`设置为`False`）：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本。在这种情况下，您在作业规范中告诉服务模型工件的位置。
  - `--epochs`：训练的周期数。
  - `--steps`：每个周期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

DIRECT = True  # Set False to use AIP_MODEL_DIR
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

运行自定义训练任务

接下来，您可以通过调用`run()`方法来运行自定义任务，启动训练作业，并传入以下参数:

- `args`: 传递给训练脚本的命令行参数。
- `replica_count`: 训练所需的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`: 计算实例的机器类型。
- `accelerator_type`: 硬件加速器的类型。
- `accelerator_count`: 要连接到工作 replica 的加速器数量。
- `base_output_dir`: 用于写入模型工件的 Cloud 存储位置。
- `sync`: 设置为**True**以等待作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在，从 Cloud Storage 存储桶中加载它，然后您可以运行模型评估并为预测请求做好准备。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

### 加载评估数据

使用`tf.keras.datasets`中的`load_data()`方法加载波士顿房屋测试（留出）数据。这将返回一个包含两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和相应的标签（房主占用房屋的中位数价值）。

您不需要训练数据，因此加载为`(_, _)`。

在用于评估之前，您需要对数据进行预处理。通过将每个值除以该列的最大值，对每列中的数据进行标准化（重新缩放）。这将用32位浮点数（介于0和1之间）替换每个单个值。

In [ ]:
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(float)
    return feature


# save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(float)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

使用模型的 `evaluate()` 方法执行评估。

In [ ]:
local_model.evaluate(x_test, y_test)

## 获取服务功能的签名

通过重新加载模型到内存，并查询与每个层对应的签名，您可以获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此，您需要了解服务函数的输入层名称-- 这在您进行预测请求时会用到。

您还需要知道服务函数的输入和输出层的名称以构建解释元数据-- 这将在后续讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 解释说明规范

在进行预测时获取解释时，您必须启用解释功能并在将自定义模型上传到 Vertex AI 模型资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `参数`：这是用于解释您的模型的可解释性算法的规范。您可以在以下选项之间进行选择：
  - Shapley（**注意**：不推荐用于图像数据，因为可能需要运行很长时间）
  - XRAI
  - 综合梯度
- `元数据`：这是应用在您的自定义模型上的算法的规范。

#### 解释参数

查看关于可解释性算法设置的详细概述。

#### Shapley

将结果的功劳分配给每个特征，并考虑特征的不同排列方式。此方法提供了确切 Shapley 值的抽样近似。

使用案例：
  - 表格数据的分类和回归。

参数：

- `path_count`：这是算法处理的特征路径数。确切的 Shapley 值近似需要 M! 条路径，其中 M 是特征的数量。对于 CIFAR10 数据集，这将是 784（28*28）。

对于任何非常大数量的特征，这会非常昂贵。您可以将路径数量减少到 M * `path_count`。

#### 综合梯度

一种基于梯度的方法，能够高效计算具有与 Shapley 值相同公理属性的特征归因。

使用案例：
  - 表格数据的分类和回归。
  - 图像数据的分类。

参数：

- `step_count`：这是用于近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域，创建出一张突出显示图像相关区域而不是像素的显著性地图。

使用案例：
  - 图像数据的分类。

参数：

- `step_count`：这是用于近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是 50 步，但随着步数增加，计算时间也会增加。

在下一个代码单元格中，将变量 `XAI` 设置为您在自定义模型中使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

#### 解释元数据

解释元数据包括：

- `outputs`：输出的标量值 -- 需要解释的部分。例如，在分类的概率输出 \[0.1, 0.2, 0.7\] 中，想要解释 0.7。考虑以下公式，其中输出为 `y`，需要解释。

    y = f(x)

考虑以下公式，其中输出为 `y` 和 `z`。由于只能为一个标量值进行归因，你必须选择是要解释输出 `y` 还是 `z`。在这个例子中，假设模型是目标检测，y 和 z 分别是边界框和物体分类。你需要选择要解释的两个输出之一。

    y, z = f(x)

`outputs` 字典格式如下：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释的输出分配的可读名称。一个常见的例子是 "概率"。<br/>
 -  "output_tensor_name": 用于识别要解释的输出层的键/值字段。<br/>
 -  [layer]: 要解释的输出层。在单任务模型中，比如表格回归器，它是模型中最后（最顶层）的层。
</blockquote>

- `inputs`：属性的特征 -- 它们对输出的贡献。考虑以下公式，其中 `a` 和 `b` 是特征。你需要选择要解释如何贡献的特征。假设这个模型用于 A/B 测试，其中 `a` 是用于预测的数据项，`b` 用于识别模型实例是 A 还是 B。你需要选择 `a`（或其一部分）作为特征，而不选择 `b`，因为它对预测没有贡献。

    y = f(a,b)

`inputs` 的最小字典格式如下：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释的输入分配的可读名称。一个常见的例子是 "特征"。<br/>
 -  "input_tensor_name": 用于识别特征贡献的输入层的键/值字段。<br/>
 -  [layer]: 用于特征贡献的输入层。在单输入张量模型中，它是模型中第一个（最底层）的层。
</blockquote>

由于模型的输入是表格形式的，你可以指定以下两个额外字段以作为报告/可视化的辅助：

<blockquote>
 - "modality": "image": 表示字段值是图像数据。
</blockquote>

由于模型的输入是表格形式的，你可以指定以下两个额外字段以作为报告/可视化的辅助：

<blockquote>
 - "encoding": "BAG_OF_FEATURES" : 表示输入是一组表格特征。<br/>
 - "index_feature_mapping": [ feature-names ] : 每个特征的人类可读名称列表。在这个例子中，必须使用数据集中指定的特征名称。<br/>
 - "modality": "numeric": 表示字段值是数字。
</blockquote>

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aiplatform.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aiplatform.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aiplatform.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到Vertex AI模型注册表，使用以下参数：

- `display_name`：模型资源的可读名称。
- `artifact`：经过训练的模型工件的Cloud Storage位置。
- `serving_container_image_uri`：服务容器镜像。
- `sync`：是否异步或同步执行上传。
- `explanation_parameters`：配置解释模型预测的参数。
- `explanation_metadata`：描述用于解释模型输入和输出的元数据。

如果`upload()`方法以异步方式运行，您随后可以使用`wait()`方法阻塞并等待完成。

In [ ]:
model = aiplatform.Model.upload(
    display_name="boston",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

## 部署模型

接下来，为在线预测部署您的模型。要部署模型，请调用`deploy()`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的人类可读名称。
- `traffic_split`：发送至端点的流量百分比，作为一个或多个键/值对的字典指定。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"表示上传的这个模型，100意味着100%的流量。
如果有现有模型在端点上，如果要将流量拆分，使用 model_id 指定为 { "0": 百分比，model_id: 百分比, ... }，其中 model_id 是将要部署到端点的现有模型的模型标识。百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作副本的加速器数量。
- `starting_replica_count`：初始预配的计算实例数。
- `max_replica_count`：要扩展到的最大计算实例数。在本教程中，仅预配了一个实例。

In [ ]:
DEPLOYED_NAME = "boston"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

获取测试项目

从数据集的测试（留出）部分中使用一个示例作为测试项目。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 生成带有解释的预测

现在你的模型已部署到端点，你可以通过向端点发送预测请求来获得在线解释。

#### 请求

每个实例的格式为：

    [特征列表]

由于`explain()`方法可以接受多个项（实例），请将你的单个测试项目作为一个测试项目列表发送。

#### 响应

从`explain()`调用中返回的响应是一个包含以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个实例的预测。
- `deployed_model_id`：生成预测的部署模型资源的Vertex AI标识符。
- `explanations`：可解释AI返回的特征归因。

In [ ]:
instances_list = [test_item.tolist()]

prediction = endpoint.explain(instances_list)
print(prediction)

理解解釋回答

首先，看看您的模型預測的結果，並將其與實際值進行比較。

In [ ]:
value = prediction[0][0][0]
print("Predicted Value:", value)

### 检查特征归因

查看此特定示例的特征归因。 正属性值意味着某个特征将您的模型预测提高了该数量，负属性值则相反。

In [ ]:
from tabulate import tabulate

feature_names = [
    "crim",
    "zn",
    "indus",
    "chas",
    "nox",
    "rm",
    "age",
    "dis",
    "rad",
    "tax",
    "ptratio",
    "b",
    "lstat",
]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, test_item[i], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查您的解释和基准线

为了更好地理解您收到的特征归因，您应该将它们与您模型的基线进行比较。在大多数情况下，您的归因值和基线的总和应该非常接近您模型对每个输入的预测值。还要注意，对于回归模型，从解释返回的`baseline_score`对于发送到您模型的每个示例都保持不变。对于分类模型，每个类都有自己的基线。

在本节中，发送10个测试示例到您的模型进行预测，以便将特征归因与基线进行比较。然后，通过`sanity_check_explanations()`函数对每个测试示例的归因进行一次检查。

#### 获取解释

In [ ]:
# Prepare 10 test examples to your model for prediction
instances = []
for i in range(10):
    instances.append(x_test[i].tolist())

response = endpoint.explain(instances)

#### 健康检查

在下面的功能中，您对解释进行健康检查。

In [ ]:
def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

清理

要清理本项目中使用的所有Google Cloud资源，您可以[删除您用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy the model from the endpoint
endpoint.undeploy_all()

# Delete the model endpoint
endpoint.delete()

# Delete the model resource
model.delete()

# Delete the custom training job
job.delete()

# Delete the locally generated files
! rm -rf custom custom.tar.gz

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI